In [8]:
import os
from pathlib import Path

import pandas as pd

from src.config import DataFoldersConfig

os.chdir(r"C:\Users\Vladyslav_Synytsia\my_projects\to_be_named")

Read all dataframes into a list

In [9]:
raw_data_folder = Path(DataFoldersConfig.RAW)
dfs = [
    pd.read_csv(csv_file)
    for csv_file in raw_data_folder.glob("*.csv")
]

Concatenate all data frames

In [10]:
all_data = pd.concat(dfs, ignore_index=True)
all_data.isna().sum()

url               0
artist_name     617
label           669
album_name      617
genre          2919
summary         870
review_text     618
score             0
dtype: int64

Deal with NaN values

In [11]:
nan_columns = all_data.columns[all_data.isna().any()].tolist()
all_data = all_data.dropna(subset=nan_columns, how="all")

all_data["summary"] = all_data["summary"].fillna("")
all_data = all_data.dropna()

Remove redundant text from reviews

In [13]:
def remove_affiliate_text(string: str) -> str:
    affiliate_text = "However, when you buy something through our retail links"
    idx_start = string.find(affiliate_text)
    return string[:idx_start].rstrip()

all_data.review_text = all_data.review_text.apply(remove_affiliate_text)

In [14]:
all_data["text"] = all_data.apply(lambda row: row["summary"] + row["review_text"], axis=1)
all_data = all_data.drop(columns=["summary", "review_text"])

In [15]:
preprocessed_data_folder = Path(DataFoldersConfig.PREPROCESSED)
fname = preprocessed_data_folder / "reviews.csv"
all_data.to_csv(fname, index=False)